In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric
sns.set(font="Malgun Gothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')


In [4]:
Data= pd.read_excel('(2022-11-21)_왓섭.xlsx')
Data=Data[Data['별점']==5]

## LDA
참고: https://happy-obok.tistory.com/5 / https://dacon.io/competitions/official/235590/codeshare/1078

In [7]:
from gensim import corpora

In [8]:
#특정 열의 값 별 몇 개의 행이 있는지 세기

element_count = {}

for item in Data['별점']:
    element_count.setdefault(item,0)
    element_count[item] += 1
    
count = pd.DataFrame.from_dict(element_count, orient = 'index',columns=["count"])
count

,count
5,205


In [9]:
#데이터 프레임의 'text' 열의 값들을 str 형식으로 바꾸기
Data.별점 = Data.별점.astype(str)

#text와 timestamp 열을 기준으로 중복된 데이터를 삭제, inplace : 데이터 프레임을 변경할지 선택(원본을 대체)
#Data.drop_duplicates(subset=['text','timestamp'], inplace=True)

#한글이 아니면 빈 문자열로 바꾸기
Data['리뷰내용'] = Data['리뷰내용'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ',regex=True)

#빈 문자열 NAN 값으로 바꾸기
Data = Data.replace({'': np.nan})
Data = Data.replace(r'^\s*$', None, regex=True)

#NAN 이 있는 행은 삭제
Data.dropna(how='any', inplace=True)

#인덱스 차곡차곡
Data = Data.reset_index (drop = True)

#데이터 프레임에 null 값이 있는지 확인
print(Data.isnull().values.any()) 

False


In [10]:
#텍스트 데이터를 리스트로 변환
Data_list=Data['리뷰내용'].values.tolist()

In [11]:
#pip install konlpy
#konlpy 설치

In [12]:
from konlpy.tag import Kkma
kkma=Kkma()
#https://mr-doosun.tistory.com/22

In [13]:
#텍스트 데이터를 리스트로 변환
Data_list=Data.리뷰내용.values.tolist()
#리스트를 요소별로(트윗 하나) 가져와서 명사만 추출한 후 리스트로 저장
data_word=[]
for i in range(len(Data_list)):
    try:
        data_word.append(kkma.nouns(Data_list[i]))
    except Exception as e:
        continue

#data_word

In [14]:
#pip install gensim

from collections import Counter
from konlpy.tag import Okt
okt = Okt()
%matplotlib inline
import matplotlib.pyplot as plt
import re 
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel 

mallet 설정: https://github.com/RaRe-Technologies/gensim/issues/2163

In [15]:
import os
os.environ.update({'MALLET_HOME':r'C:/mallet-2.0.8/'})

mallet_path = 'C:/mallet-2.0.8/bin/mallet'

In [16]:
id2word=corpora.Dictionary(data_word)
id2word.filter_extremes(no_below = 3) #3회 이하로 등장한 단어는 삭제
texts = data_word
corpus=[id2word.doc2bow(text) for text in texts]

#mallet_path = 'C:\\mallet-2.0.8\\bin\\mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)

In [17]:
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()


def compute_coherence_values(dictionary, corpus, texts, limit, start=4, step=2):

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=data_word, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [18]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=4, limit=21, step=2)

In [19]:

limit=21; start=4; step=2;
x = range(start, limit, step)
topic_num = 0
count = 0
max_coherence = 0
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", cv)
    coherence = cv
    if coherence >= max_coherence:
        max_coherence = coherence
        topic_num = m
        model_list_num = count   
    count = count+1

        
# Select the model and print the topics
optimal_model = model_list[model_list_num]
model_topics = optimal_model.show_topics(formatted=False)
#print(optimal_model.print_topics(num_words=10))








def format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    #ldamodel[corpus]: lda_model에 corpus를 넣어 각 토픽 당 확률을 알 수 있음
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num,topn=10)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    print(type(sent_topics_df))

    # Add original text to the end of the output
    #contents = pd.Series(texts)
    #sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df = pd.concat([sent_topics_df, Data['리뷰내용'],Data['별점']], axis=1)
    return(sent_topics_df)



Num Topics = 4  has Coherence Value of 0.440955018176206
Num Topics = 6  has Coherence Value of 0.4846533610568417
Num Topics = 8  has Coherence Value of 0.5569005706760448
Num Topics = 10  has Coherence Value of 0.5593947511401083
Num Topics = 12  has Coherence Value of 0.5571978582436409
Num Topics = 14  has Coherence Value of 0.563285675565644
Num Topics = 16  has Coherence Value of 0.587024471481259
Num Topics = 18  has Coherence Value of 0.5864388406686643
Num Topics = 20  has Coherence Value of 0.5791274379439552


In [20]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=Data_list)

# Format
df_topic_tweet = df_topic_sents_keywords.reset_index()
df_topic_tweet.columns = ['Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords','', '리뷰내용','별점']

# Show각 문서에 대한 토픽
#df_dominant_topic=df_dominant_topic.sort_values(by=['Dominant_Topic'])
#df_topic_tweet



# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)


topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
topic_counts.sort_index(inplace=True)

topic_contribution = round(topic_counts/topic_counts.sum(), 4)
topic_contribution

lda_inform = pd.concat([sent_topics_sorteddf_mallet, topic_counts, topic_contribution], axis=1)
lda_inform.columns=["Topic_Num", "Topic_Perc_Contrib", "Keywords", '리뷰내용','별점',"Num_Documents", "Perc_Documents"]
lda_inform = lda_inform[["Topic_Num","Keywords","Num_Documents","Perc_Documents"]]
lda_inform
#lda_inform.Topic_Num = lda_inform.Topic_Num.astype(int)
lda_inform['Topic_Num'] =lda_inform['Topic_Num'] +1
lda_inform.Topic_Num = lda_inform.Topic_Num.astype(str)
lda_inform['Topic_Num'] =lda_inform['Topic_Num'].str.split('.').str[0]
df_topic_tweet['Dominant_Topic'] =df_topic_tweet['Dominant_Topic'] +1
df_topic_tweet.Dominant_Topic = df_topic_tweet.Dominant_Topic.astype(str)
df_topic_tweet['Dominant_Topic'] =df_topic_tweet['Dominant_Topic'].str.split('.').str[0]

<class 'pandas.core.frame.DataFrame'>


In [37]:
lda_inform.to_csv ("lda_inform5.csv",encoding='ANSI', index = None)
lda_inform

,Topic_Num,Keywords,Num_Documents,Perc_Documents
0.0,1,"관리, 왓섭, 고객, 이거, 무료, 왓섭으, 해지, 여기저기, 등록, 관련",29,0.1415
1.0,2,"한눈, 추천, 유용, 생활, 금액, 제가, 완전, 정, 리뷰, 어플",20,0.0976
2.0,3,"돈, 확인, 내, 안내, 시간, 이체, 일, 덕분, 꿀, 설명",16,0.0780
3.0,4,"가능, 연동, 구독서비스, 눈, 이용, 카카오, 도움, 신경, 다운, 페",10,0.0488
4.0,5,"친절, 사용, 상담, 문제, 체크, 센터, 고객센터, 뿐, 고객, 불편",17,0.0829
5.0,6,"앱, 분, 처리, ㅎ, 하나, 저, 입력, 직관적, 전, 방법",15,0.0732
6.0,7,"지출, 한번, 고정, 추가, 월, 통신비, 달, 요즘, 정확, 응대",16,0.0780
7.0,8,"구독, 안, 정보, 구, 자동이체, 계좌, 시, 개, 필요, 생각",6,0.0293
8.0,9,"정기, 정기결제, 거, 다, 만족, 상담, 번호, 필요, 피드백, 페",11,0.0537
9.0,10,"감사, 거, 결제, 내역, 기능, 가입, 독, 중인, 디자인, 데",9,0.0439


## 시각화

In [22]:
#pip install wordcloud-1.7.0-cp38-cp38-win_amd64.whl


In [23]:
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [36]:
data_word

[['장점',
  '소액',
  '지출',
  '수',
  '거',
  '달',
  '체크',
  '가능',
  '가계부',
  '가계부등',
  '등',
  '지출내역',
  '내역',
  '정리',
  '때',
  '유용',
  '처음',
  '로그',
  '로그인',
  '인',
  '인증',
  '문의',
  '응대',
  '친절',
  '서비스',
  '해결',
  '단점',
  '부분',
  '있닐',
  '있닐대',
  '대',
  '시간',
  '약간',
  '꺼',
  '문제',
  '플'],
 ['디자인',
  '기능',
  '생각',
  '광고',
  '광고보고',
  '보고',
  '거',
  '거라',
  '라',
  '연동',
  '정기',
  '정기결제',
  '결제',
  '많으',
  '예상',
  '외의',
  '기능성',
  '로딩',
  '초',
  '초',
  '한국인',
  '불편',
  '로딩속',
  '전반적',
  '만족',
  '보안',
  '신경'],
 ['안녕',
  '플',
  '직관적',
  '고객',
  '고객경험개선',
  '경험',
  '개선',
  '도움',
  '처음',
  '공인',
  '공인인증서',
  '인증서',
  '로그',
  '로그인',
  '인',
  '시도',
  '때',
  '계속',
  '비밀',
  '비밀번호',
  '번호',
  '오류',
  '구',
  '짜증',
  '하나',
  '은행',
  '은행앱',
  '앱',
  '이상',
  '영문',
  '입력',
  '입력간',
  '간',
  '대문자',
  '거',
  '그거',
  '때문',
  '은행어플',
  '별도',
  '입력기',
  '기'],
 ['다',
  '동일',
  '계좌',
  '보험',
  '자동',
  '자동이체',
  '이체',
  '지금',
  '둘',
  '중',
  '건',
  '세부',
  '내역',
  '확인',
  '수',
  '이것',
  '합산',
  '금액',
  '럭',
  '

In [33]:
processed_data=data_word
num_topics = 3
chunksize = 300
passes = 20
iterations = 2
eval_every = None
dictionary = corpora.Dictionary(processed_data)
temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [34]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'file_name_5.html')

In [26]:
#pip install pyLDAvis